In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
import numpy as np

In [3]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [4]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


/Users/aa406165/Git/qiskit-core/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [5]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [6]:
from qiskit.providers.aer import AerSimulator

backend = AerSimulator()

In [7]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shos=1000)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueResult(value=1.767578125, variance=11.192073822021484, confidence_interval=(1.6079491702827564, 1.9272070797172436))

In [8]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueResult(value=2.044921875, variance=10.808177947998047, confidence_interval=(1.8901196575140042, 2.199724092485996))

In [9]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueArrayResult(values=array([1.85133333, 0.132     ]), variances=array([11.01174356, 10.358456  ]), confidence_intervals=array([[1.75947564, 1.94319103],
       [0.03896703, 0.22503297]]))

In [10]:
# can pass ndarray
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]), shots=8192)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


ExpectationValueArrayResult(values=array([1.83886719, 0.18041992]), variances=array([11.0697484 , 10.49843544]), confidence_intervals=array([[1.78300835, 1.89472603],
       [0.12375936, 0.23708049]]))

### Exact simulation by SaveExpectationValueVariance

In [11]:
from qiskit.evaluators import ExactExpectationValue

In [12]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.8420921273153708, variance=6.432763522806668, confidence_interval=None)

In [13]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]))

ExpectationValueArrayResult(values=array([1.84209213, 0.15611897]), variances=array([6.43276352, 5.35681431]), confidence_intervals=array([None, None], dtype=object))

### Transpiled Circuits

In [14]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.set_transpile_options(initial_layout=[3, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
                                                                »
ancilla_0 -> 0 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 1 ─────────────────────────────────────────────────»
               ┌────┐┌──────────────┐┌────┐┌────────┐┌───┐┌────┐»
      q_1 -> 2 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤└─┬─┘├────┤»
      q_0 -> 3 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘     └────┘»
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c4: 2/═════════════════════════════════════════════════»
                                                                »
«                                                                          »
«ancilla_0 -> 0 ─────────────────────────────────

### Transpile options and Run options

In [15]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=100_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.1805337519623236, variance=12.78985270451538, confidence_interval=(1.1625961385375756, 1.1984713653870716))

In [16]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.184814453125, variance=12.772483766078949, confidence_interval=(1.1222021421089112, 1.2474267641410888))

In [17]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=1.4266666666666665, variance=12.511555555555555, confidence_interval=(1.1038181559888551, 1.749515177344478))

### Composite Evaluator

In [18]:
from qiskit.evaluators import JointEvaluator

In [19]:
len(expval.transpiled_circuits)

3

In [20]:
joint_evaluator = JointEvaluator([expval, expval, expval])  # can be different evaluator

In [21]:
len(joint_evaluator.transpiled_circuits)  # 3 × 3

9

In [22]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=1.10107421875, variance=12.911907315254211, confidence_interval=(1.038012240246375, 1.164136197253625)), ExpectationValueResult(value=1.10107421875, variance=12.911907315254211, confidence_interval=(1.038012240246375, 1.164136197253625)), ExpectationValueResult(value=1.10107421875, variance=12.911907315254211, confidence_interval=(1.038012240246375, 1.164136197253625))])

### Readout error mitigation

In [23]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(
    backend, mitigation="tensored", refresh=600, shots=2000, mit_pattern=[[0], [1]]
)
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(
    f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)
print(
    f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)

w/o mitigation shots=2000, result=ExpectationValueResult(value=1.2100000000000002, variance=12.667566, confidence_interval=(1.0839861116045295, 1.3360138883954709))
w/  mitigation shots=2000, result=ExpectationValueResult(value=1.3306489073822014, variance=12.386664719334453, confidence_interval=(1.2066056176880575, 1.4546921970763453))


### Gradient of expectation value

In [24]:
from qiskit.evaluators.expectation_value.expectation_value_gradient import (
    FiniteDiffGradient,
    ParameterShiftGradient,
)

In [25]:
exact_expval = ExactExpectationValue(ansatz, observable, backend=AerSimulator())
exact_findiff = FiniteDiffGradient(exact_expval, 1e-8)
print(f"fin diff of exact {exact_findiff.evaluate([0, 1, 1, 2, 3, 5]).values}")

shots = 2000
findiff = FiniteDiffGradient(expval_raw, 1e-1)
paramshift = ParameterShiftGradient(expval_raw)
print(f"fin diff w/o mit {findiff.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")
print(
    f"param shift w/o mit {paramshift.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}"
)

findiff = FiniteDiffGradient(expval_mit, 1e-1)
paramshift = ParameterShiftGradient(expval_mit)
print(f"fin diff w/  mit {findiff.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}")
print(
    f"param shift w/  mit {paramshift.evaluate([0, 1, 1, 2, 3, 5], shots=shots).values}"
)

fin diff of exact [0.13510324 1.87744373 1.03215423 1.87744376 1.48950676 2.15662823]
fin diff w/o mit [-0.71  1.01  0.26  1.16 -0.5  -0.39]
param shift w/o mit [0.136  1.1595 0.7    1.1715 0.943  1.356 ]
fin diff w/  mit [0.33002762 1.4563516  0.98946664 1.68256132 0.65200418 0.82321132]
param shift w/  mit [0.13989401 1.46481847 0.90215406 1.4520251  1.17709897 1.57126144]


### VQE by Scipy optimizer

In [26]:
from scipy.optimize import minimize

shot = 10000
backend = AerSimulator()
expval = PauliExpectationValue(ansatz, observable, backend=backend)
paramshift = ParameterShiftGradient(expval)
# this may take a long time...
result = minimize(
    lambda x: expval_mit.evaluate(x, shots=shots).value,
    np.zeros(6),
    jac=lambda x: paramshift.evaluate(x, shots=shots).values,
)
print(result)

no max_experiments for backend 'aer_simulator'. Set 1000000 as max_experiments.


      fun: -4.602087177884175
 hess_inv: array([[ 3.68145975e-01,  4.51919660e-03, -2.97237170e-01,
         3.40287647e-01, -6.22648878e-02, -1.14357822e-01],
       [ 4.51919660e-03,  2.33167219e-01, -1.63323961e-02,
         2.83679348e-02, -2.54342635e-02,  1.82316849e-02],
       [-2.97237170e-01, -1.63323961e-02,  6.46774967e-01,
        -5.44220002e-01,  1.04814033e-02,  1.45380517e-01],
       [ 3.40287647e-01,  2.83679348e-02, -5.44220002e-01,
         7.51195003e-01,  1.22922753e-04, -1.49624548e-01],
       [-6.22648878e-02, -2.54342635e-02,  1.04814033e-02,
         1.22922753e-04,  4.77940615e-01,  3.09647535e-01],
       [-1.14357822e-01,  1.82316849e-02,  1.45380517e-01,
        -1.49624548e-01,  3.09647535e-01,  3.20785074e-01]])
      jac: array([-0.044 ,  0.078 , -0.004 , -0.0285,  0.1265, -0.0535])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 43
      nit: 12
     njev: 31
   status: 2
  success: False
        x: array([-1.971